In [18]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import cross_validation, metrics
from sklearn.grid_search import GridSearchCV

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

# 读取数据

In [62]:

train_url="/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/train.csv"
train=pd.read_csv(train_url)

test_url="/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/test.csv"
test=pd.read_csv(test_url)


# 数据探查

In [63]:
print "train shape:",train.shape
print "test shape:",test.shape
print "train columns:",train.columns


train shape: (595212, 59)
test shape: (892816, 58)
train columns: Index([u'id', u'target', u'ps_ind_01', u'ps_ind_02_cat', u'ps_ind_03',
       u'ps_ind_04_cat', u'ps_ind_05_cat', u'ps_ind_06_bin', u'ps_ind_07_bin',
       u'ps_ind_08_bin', u'ps_ind_09_bin', u'ps_ind_10_bin', u'ps_ind_11_bin',
       u'ps_ind_12_bin', u'ps_ind_13_bin', u'ps_ind_14', u'ps_ind_15',
       u'ps_ind_16_bin', u'ps_ind_17_bin', u'ps_ind_18_bin', u'ps_reg_01',
       u'ps_reg_02', u'ps_reg_03', u'ps_car_01_cat', u'ps_car_02_cat',
       u'ps_car_03_cat', u'ps_car_04_cat', u'ps_car_05_cat', u'ps_car_06_cat',
       u'ps_car_07_cat', u'ps_car_08_cat', u'ps_car_09_cat', u'ps_car_10_cat',
       u'ps_car_11_cat', u'ps_car_11', u'ps_car_12', u'ps_car_13',
       u'ps_car_14', u'ps_car_15', u'ps_calc_01', u'ps_calc_02', u'ps_calc_03',
       u'ps_calc_04', u'ps_calc_05', u'ps_calc_06', u'ps_calc_07',
       u'ps_calc_08', u'ps_calc_09', u'ps_calc_10', u'ps_calc_11',
       u'ps_calc_12', u'ps_calc_13', u'ps_calc_14

0    573518
1     21694
Name: target, dtype: int64

In [68]:
target='target'
IDcol = 'id'

train['target'].value_counts()

0    573518
1     21694
Name: target, dtype: int64

In [71]:
train_pos=train[train['target']==1]
train_pos['target'].value_counts()

1    21694
Name: target, dtype: int64

In [64]:
train.describe()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,...,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,743803.558435,0.036448,1.900378,1.358943,4.423318,0.416794,0.405188,0.393742,0.257033,0.163921,...,5.441382,1.441918,2.872288,7.539026,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,429367.820429,0.187401,1.983789,0.664594,2.699902,0.493311,1.350642,0.488579,0.436998,0.370205,...,2.332871,1.202963,1.694887,2.746652,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,7.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,371991.500000,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,743547.500000,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,1115548.750000,0.000000,3.000000,2.000000,6.000000,1.000000,0.000000,1.000000,1.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1488027.000000,1.000000,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,...,19.000000,10.000000,13.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [65]:
train.isnull().any()

id                False
target            False
ps_ind_01         False
ps_ind_02_cat     False
ps_ind_03         False
ps_ind_04_cat     False
ps_ind_05_cat     False
ps_ind_06_bin     False
ps_ind_07_bin     False
ps_ind_08_bin     False
ps_ind_09_bin     False
ps_ind_10_bin     False
ps_ind_11_bin     False
ps_ind_12_bin     False
ps_ind_13_bin     False
ps_ind_14         False
ps_ind_15         False
ps_ind_16_bin     False
ps_ind_17_bin     False
ps_ind_18_bin     False
ps_reg_01         False
ps_reg_02         False
ps_reg_03         False
ps_car_01_cat     False
ps_car_02_cat     False
ps_car_03_cat     False
ps_car_04_cat     False
ps_car_05_cat     False
ps_car_06_cat     False
ps_car_07_cat     False
ps_car_08_cat     False
ps_car_09_cat     False
ps_car_10_cat     False
ps_car_11_cat     False
ps_car_11         False
ps_car_12         False
ps_car_13         False
ps_car_14         False
ps_car_15         False
ps_calc_01        False
ps_calc_02        False
ps_calc_03      

## Define a function for modeling and cross-validation

This function will do the following:
1. fit the model
2. determine training accuracy
3. determine training AUC
4. determine testing AUC
5. update n_estimators with cv function of xgboost package
6. plot Feature Importance 

## Step 1- Find the number of estimators for a high learning rate

In [22]:
predictors = [x for x in train.columns if x not in [target, IDcol]]
X = train[predictors]
y = train['target']
X_test = test[predictors]
gbm0 = GradientBoostingClassifier(random_state=10)
gbm0.fit(X,y)
y_pred = gbm0.predict(X)
y_predprob = gbm0.predict_proba(X)[:,1]
print "Accuracy : %.4g" % metrics.accuracy_score(y.values, y_pred)
print "AUC Score (Train): %f" % metrics.roc_auc_score(y, y_predprob)

Accuracy : 0.9636
AUC Score (Train): 0.646436


In [66]:
from sklearn.metrics import roc_auc_score
def gini(y_actual, y_pred):
  return 2*roc_auc_score(y_actual, y_pred)-1


In [67]:
#GINI score
print "GINI Score (Train): %f" % gini(y, y_predprob)

GINI Score (Train): 0.292873


In [37]:
y_test_pred = gbm0.predict(X_test)

y_test_predprob = gbm0.predict_proba(X_test)[:,1]


In [38]:
df_y_test_predprob=pd.DataFrame([y_test_predprob])

In [39]:
X_ID = test['id']
df_y_test_predprob=df_y_test_predprob.T
print df_y_test_predprob.shape,X_ID.shape

(892816, 1) (892816,)


In [40]:
df_y_test_predprob.head(10)

,0
0,0.025462
1,0.028185
2,0.030646
3,0.016758
4,0.037023
5,0.037788
6,0.020662
7,0.037953
8,0.062781
9,0.059089


In [41]:
X_ID.head(10)

0     0
1     1
2     2
3     3
4     4
5     5
6     6
7     8
8    10
9    11
Name: id, dtype: int64

In [54]:
result=pd.concat([X_ID,df_y_test_predprob],axis=1)
result.shape

(892816, 2)

In [55]:
result.head(10)

,id,0
0,0,0.025462
1,1,0.028185
2,2,0.030646
3,3,0.016758
4,4,0.037023
5,5,0.037788
6,6,0.020662
7,8,0.037953
8,10,0.062781
9,11,0.059089


In [56]:
result.columns = ['id', 'target']
result.head(10)

,id,target
0,0,0.025462
1,1,0.028185
2,2,0.030646
3,3,0.016758
4,4,0.037023
5,5,0.037788
6,6,0.020662
7,8,0.037953
8,10,0.062781
9,11,0.059089


In [59]:
result.to_csv("/Users/leidelong/competition/Porto_Seguro_Safe_Driver_Prediction/test_result.csv",index=False)